In [10]:
import numpy as np
from state_of_the_art.tables.insights_table import InsightsTable
InsightsTable

df = InsightsTable().read().sort_values(by='tdw_timestamp', ascending=False)
df = df[df['score'].notnull()]

df
len(df.index)


48

In [11]:

rows_number=len(df) 
test_size = int(rows_number*0.3)
train_size = rows_number - test_size

train = df.sample(train_size, random_state=42)
df_all = df.merge(train.drop_duplicates(), on=['tdw_uuid','tdw_uuid'],
                   how='left', indicator=True, suffixes=('', '_y'))

#create DataFrame with rows that exist in first DataFrame only
test = df_all[df_all['_merge'] == 'left_only']
test = test[[c for c in test.columns if not c.endswith('_y')]]
# drop merge column
test = test.drop(columns=['_merge'])
train, test

(                                             insight  \
 0  This paper aims to enhance Marketing Mix Model...   
 0  Addressing ethical concerns in AI involves int...   
 0  This paper addresses the challenge of accurate...   
 0  Media Mix Modeling (MMM), widely used for eval...   
 0  Instrumental Variables (IV): Variables used in...   
 0  It addresses key challenges such as causal het...   
 0  Utilizing a data-to-deployment approach is ess...   
 0  The complex architecture and large parameter s...   
 0  Apply causal mediation analysis across various...   
 0  Potential challenges in the practical deployme...   
 0                                              arXiv   
 0  Highlights the importance of discovering more ...   
 0  Encourage interdisciplinary collaboration to a...   
 0  The establishment of ethical AI systems must e...   
 0  Traditional methods of measuring advertising e...   
 0  Data Science generates significant value in bo...   
 0  Compute-optimal model: Refe

In [19]:
import numpy as np


train_y = train['score'].astype('int64').to_numpy()
test_y =  test['score'].astype('int64').to_numpy()
train_y

array([3, 0, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 0, 3, 1, 1, 3, 3, 3, 2, 1,
       1, 0, 3, 1, 1, 1, 3, 1, 2, 3, 3, 1])

In [20]:

from sentence_transformers import SentenceTransformer

# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-mpnet-base-v2")

# The sentences to encode
# 2. Calculate embeddings by calling model.encode()
train_embedding = embeddings = model.encode(train['insight'].to_list())
test_embedding = embeddings = model.encode(test['insight'].to_list())
train_embedding[0]

array([ 1.63112022e-02,  5.40993437e-02, -4.27001268e-02, -7.86587298e-02,
        1.87284704e-02, -7.04803364e-03, -7.46028079e-03, -5.35690524e-02,
       -3.14531289e-02,  5.13109379e-03,  4.81660627e-02,  7.99831003e-03,
        3.29758413e-03,  1.08359255e-01,  1.28984880e-02, -6.25553681e-03,
       -1.12415496e-02, -1.28961084e-02, -1.18942866e-02,  1.85619127e-02,
        2.75384281e-02, -1.89171676e-02, -1.54445479e-02,  3.92184965e-03,
       -3.61229405e-02,  1.71571895e-02, -7.21734017e-03, -1.21270062e-03,
        4.70614014e-03, -2.46885363e-02,  1.91318300e-02, -3.65155074e-03,
        5.37413172e-02, -8.19978584e-03,  2.24007749e-06, -2.21518483e-02,
       -5.83249666e-02,  1.79918353e-02, -5.04939072e-02,  9.73391440e-03,
        3.60296108e-02,  5.89034744e-02,  2.38524545e-02,  2.17687581e-02,
       -2.46361829e-03,  3.47468667e-02,  1.69534218e-02,  1.01124011e-02,
       -3.29750478e-02,  1.56299341e-02, -3.13285179e-02, -2.75733545e-02,
       -3.12706344e-02, -

In [21]:

import torch
from torch import nn

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(768, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 5)
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()
print(model)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=768, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=5, bias=True)
  )
)


In [22]:
def train_fn(dataloader, model, loss_fn, optimizer):
    size = len(dataloader)

    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X = torch.from_numpy(X)
        y = torch.from_numpy(np.array(y))

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [23]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
epochs = 2000

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_fn(list(zip(train_embedding, train_y)), model, loss_fn, optimizer)
print("Done!")

Epoch 1
-------------------------------
loss: 1.554577  [  768/   34]
Epoch 2
-------------------------------
loss: 1.549155  [  768/   34]
Epoch 3
-------------------------------
loss: 1.543832  [  768/   34]
Epoch 4
-------------------------------
loss: 1.538610  [  768/   34]
Epoch 5
-------------------------------
loss: 1.533482  [  768/   34]
Epoch 6
-------------------------------
loss: 1.528471  [  768/   34]
Epoch 7
-------------------------------
loss: 1.523574  [  768/   34]
Epoch 8
-------------------------------
loss: 1.518784  [  768/   34]
Epoch 9
-------------------------------
loss: 1.514055  [  768/   34]
Epoch 10
-------------------------------
loss: 1.509418  [  768/   34]
Epoch 11
-------------------------------
loss: 1.504871  [  768/   34]
Epoch 12
-------------------------------
loss: 1.500414  [  768/   34]
Epoch 13
-------------------------------
loss: 1.496035  [  768/   34]
Epoch 14
-------------------------------
loss: 1.491742  [  768/   34]
Epoch 15
------

In [24]:
model(torch.from_numpy(test_embedding[4]))

tensor([ 0.2582, -4.4294,  1.9105,  5.1808, -2.4862], grad_fn=<ViewBackward0>)

In [25]:
test

,insight,paper_id,score,tdw_timestamp,tdw_uuid,question
7,Focus on developing domain-specific adaptation...,https://arxiv.org/abs/2408.01319,1.0,2024-08-06 05:27:34.165891,727b0973-9ea6-453a-9486-700907439091,top_recommended_actions
10,This paper underscores the essential role of c...,https://arxiv.org/abs/1907.07892,3.0,2024-08-03 00:56:02.132973,cbcdf6f2-ad17-4739-a5cf-317e310acaa1,going_deep
14,Data Science: A multidisciplinary field that u...,https://arxiv.org/abs/2201.05852,1.0,2024-08-03 00:39:54.366042,30005ff8-6a03-4e73-8887-c89f56a500ec,core_terms_defintion
18,Direct Preference Optimization (DPO): A method...,https://arxiv.org/abs/2407.21783,3.0,2024-08-03 00:19:12.653253,d7ceaccc-6b3f-440e-8c22-6adb2a65417c,core_terms_defintion
20,"Despite significant safety measures, the risk ...",https://arxiv.org/abs/2407.21783,1.0,2024-08-03 00:19:12.635944,16ff221d-1114-47a4-bce7-1510996e0ea9,weakeness_from_paper
22,Encourage the AI research community to adopt t...,https://arxiv.org/abs/2407.21792,1.0,2024-08-02 19:15:04.611200,d590aa00-6d64-4e19-b86c-157a59ebe56e,None
23,It may overlook some qualitative aspects of sa...,https://arxiv.org/abs/2407.21792,1.0,2024-08-02 19:14:54.069890,bb971b86-bc58-482b-9f17-4288bb577f3c,None
28,The proposed CausalMMM model integrates Grange...,https://arxiv.org/abs/2406.16728,1.0,2024-08-01 09:39:44.623275,4e937ec4-2bbf-45e4-8691-a0317fac0e86,None
35,The paper demonstrates that expected marginal ...,https://arxiv.org/abs/1207.4108,3.0,2024-07-31 17:42:08.214916,e8f42bfd-3875-45a4-af37-ab0676a4b4cc,None
36,For advertisers with less competitive outside ...,https://arxiv.org/abs/2405.14279,1.0,2024-07-31 17:40:04.805800,bcc39978-fc46-4e16-8e26-aa91ae35a601,None


In [26]:

from state_of_the_art.config  import config


torch.save(model.state_dict(), config.TEXT_PREDICTOR_PATH_LOCALLY)


In [22]:
model2 =  NeuralNetwork()
model2.load_state_dict(torch.load(MODEL_PATH))
model2.eval()
model2(torch.from_numpy(test_embedding[4]))

/var/folders/xt/sg0ptwx92qn1p3709sfhc4s40000gp/T/ipykernel_87102/38184876.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model2.load_state_dict(torch.load(MODEL_PATH))


tensor([-0.0762, -0.0977,  0.1219,  0.0032, -0.0459], grad_fn=<ViewBackward0>)